# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [1]:
# ORIGINAL
import torch
import torch.nn as nn
import torchvision.models as models


class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        #resnet = models.resnet50(pretrained=True)
        resnet = models.resnet101(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad_(False)
        
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.embed = nn.Linear(resnet.fc.in_features, embed_size)

    def forward(self, images):
        features = self.resnet(images)
        features = features.view(features.size(0), -1)
        features = self.embed(features)
        return features

In [2]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision

# ATTENTION
class EncoderCNN(nn.Module):
    def __init__(self, embed_size):
        super(EncoderCNN, self).__init__()
        #resnet = models.resnet50(pretrained=True)
        resnet = models.resnet101(pretrained=True)
        for param in resnet.parameters():
            param.requires_grad_(False)
        
        modules = list(resnet.children())[:-1]
        self.resnet = nn.Sequential(*modules)
        self.embed = nn.Linear(resnet.fc.in_features, embed_size)

    def forward(self, images):
        features = self.resnet(images)
        features = features.view(features.size(0), -1)
        features = self.embed(features)
        return features
    
# ATTENTION
class BahdanauAttention(nn.Module):
    """
    Bahdanau Attention.
    Reference:
    https://blog.floydhub.com/attention-mechanism/#bahdanau-att-step1 --> Attention Mechanism
    https://github.com/sgrvinod/a-PyTorch-Tutorial-to-Image-Captioning --> PyTorch Image Captioning
    """

    def __init__(self, encoder_dim, decoder_dim, attention_dim):
        """
        :param encoder_dim: feature size of encoded images
        :param decoder_dim: size of decoder's RNN
        :param attention_dim: size of the attention network
        """
        super(BahdanauAttention, self).__init__()
        self.encoder_att = nn.Linear(encoder_dim, attention_dim)  # linear layer to transform encoded image
        self.decoder_att = nn.Linear(decoder_dim, attention_dim)  # linear layer to transform decoder's output
        self.full_att = nn.Linear(attention_dim, 1)  # linear layer to calculate values to be softmax-ed
        self.relu = nn.ReLU()
        self.tanh = nn.Tanh()
        self.softmax = nn.Softmax(dim=1)  # softmax layer to calculate weights

    def forward(self, encoder_out, decoder_hidden):
        """
        Forward propagation.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, num_pixels, encoder_dim)
        :param decoder_hidden: previous decoder output, a tensor of dimension (batch_size, decoder_dim)
        :return: attention weighted encoding, weights
        """
        att1 = self.encoder_att(encoder_out)  # (batch_size, attention_dim)
        #print('encoder_out: ', encoder_out.shape)
        #print('att1 -> (batch_size, attention_dim): ', att1.shape)
        att2 = self.decoder_att(decoder_hidden)  # (batch_size, attention_dim)
        #print('decoder_hidden: ', decoder_hidden.shape)
        #print('att2 -> (batch_size, attention_dim): ', att2.shape)
        #att = self.full_att(self.relu(att1 + att2.unsqueeze(1))).squeeze(2)  # (batch_size, num_pixels)
        att = self.full_att(self.tanh(att1 + att2))  # (batch_size, num_pixels)
        #print('att -> (batch_size, num_pixels): ', att.shape)
        alpha = self.softmax(att)  # (batch_size, num_pixels)
        #print('alpha -> (batch_size, num_pixels): ', alpha.shape)
        attention_weighted_encoding = (encoder_out * alpha.unsqueeze(2)).sum(dim=1)  # (batch_size, encoder_dim)
        #print('attention_weighted_encoding -> (batch_size, encoder_dim): ', attention_weighted_encoding.shape)

        return attention_weighted_encoding, alpha

    
class DecoderRNN(nn.Module):
    def __init__(self, embed_size, decoder_size, vocab_size, attention_size=512, encoder_size=512, dropout=0.25):
        '''
        [See the diagram of the decoder in Notebook 1]
        The RNN needs to have 4 basic components :
            1. Word Embedding layer : maps the captions to embedded word vector of embed_size.
            2. LSTM layer : inputs( embedded feature vector from CNN , embedded word vector ).
            3. Hidden layer : Takes LSTM output as input and maps it 
                          to (batch_size, caption length, decoder_size) tensor.
            4. Linear layer : Maps the hidden layer output to the number of words
                          we want as output, vocab_size.
        '''
        
        super(DecoderRNN, self).__init__()
        self.encoder_size = encoder_size
        self.attention_size = attention_size
        self.embed_size = embed_size
        self.decoder_size = decoder_size
        self.vocab_size = vocab_size
        self.dropout = dropout
        
        '''
        Embedding layer parameters:
            vocab_size : size of the dictionary of embeddings, 
                basically the number of tokens in the vocabulary(word2idx) for that batch of data.
            embed_size : the size of each embedding vector of captions
        '''
        self.embedding_layer = nn.Embedding(vocab_size, embed_size)

        '''
        BahdanauAttention layer parameters:
        '''
        self.attention_layer = BahdanauAttention(encoder_size, decoder_size, attention_size)  # attention network
        
        '''
        LSTM layer parameters :
            input_size  = embed_size 
            hidden_size = decoder_size     # number of units in hidden layer of LSTM  
            num_layers  = 1               # number of LSTM layers ( = 1, by default )
            batch_first = True            # input , output need to have batch size as 1st dimension
            dropout     = 0               # did not use dropout 
        
        Other parameters were not changed from default values provided in the PyTorch implementation.
        '''
        #print('embed_size: ', embed_size)
        #print('encoder_size: ', encoder_size)
        #print('decoder_size: ', decoder_size)
        
        # https://pytorch.org/docs/stable/generated/torch.nn.LSTMCell.html
        self.lstm_layer = nn.LSTMCell( input_size = embed_size+encoder_size, 
                             hidden_size = decoder_size, 
                             bias=True)
                             #num_layers = num_layers, 
                             #dropout = dropout, 
                             #batch_first=True )
        
        self.f_beta = nn.Linear(decoder_size, encoder_size)  # linear layer to create a sigmoid-activated gate
        self.sigmoid = nn.Sigmoid()

        self.init_h_layer = nn.Linear(encoder_size, decoder_size)  # linear layer to find initial hidden state of LSTMCell
        self.init_c_layer = nn.Linear(encoder_size, decoder_size)  # linear layer to find initial cell state of LSTMCell
        self.linear_fc_layer = nn.Linear(decoder_size, vocab_size)  # linear layer to find scores over vocabulary
        self.init_weights()  # initialize some layers with the uniform distribution

    def init_weights(self):
        """
        Initializes some parameters with values from the uniform distribution, for easier convergence.
        """
        self.embedding_layer.weight.data.uniform_(-0.1, 0.1)
        self.linear_fc_layer.bias.data.fill_(0)
        self.linear_fc_layer.weight.data.uniform_(-0.1, 0.1)


    def init_decoder_state(self, encoder_out):
        """
        Creates the initial hidden and cell states for the decoder's LSTM based on the encoded images.
        :param encoder_out: encoded images, a tensor of dimension (batch_size, encoder_dim)
        :return: hidden state, cell state
        """
        #print('encoder_out: ', encoder_out)
        #print('encoder_out.shape: ', encoder_out.shape)
        #mean_encoder_out = encoder_out.mean(dim=0) # TODO: DA SISTEMARE
        #print('mean_encoder_out: ', mean_encoder_out)
        #print('mean_encoder_out.shape: ', mean_encoder_out.shape)
        #h = self.init_h_layer(mean_encoder_out)  # (batch_size, decoder_dim)
        #c = self.init_c_layer(mean_encoder_out)
        h = self.init_h_layer(encoder_out)  # (batch_size, decoder_dim)
        c = self.init_c_layer(encoder_out)
        return h, c

    
    def forward(self, encoder_out, captions):
        '''
        Arguments :
        For a forward pass, the instantiation of the RNNDecoder class
        receives as inputs 2 arguments  :
        -> encoder_out : ouput of CNNEncoder having shape (batch_size, embed_size).
        -> captions : a PyTorch tensor corresponding to the last batch of captions 
                      having shape (batch_size, caption_length) .
        NOTE : Input parameters have first dimension as batch_size.
        '''
        
        batch_size = encoder_out.size(0)
        encoder_dim = encoder_out.size(-1)
        vocab_size = self.vocab_size
        #print ('batch_size: ', batch_size)
        #print ('encoder_out.shape: ', encoder_out.shape)
        #print ('encoder_dim: ', encoder_dim)
        #print ('vocab_size: ', vocab_size)
        #print ('captions.shape: ', captions.shape)

        # Flatten image
        #encoder_out = encoder_out.view(batch_size, -1, encoder_dim)  # (batch_size, num_pixels, encoder_dim)
        #num_pixels = encoder_out.size(1)
        #print('num_pixels: ', num_pixels)
        #print('encoder_out -> (batch_size, encoder_dim): ', encoder_out.shape)
        
        # Discard the <end> word to avoid the following error in Notebook 1 : Step 4
        # (outputs.shape[1]==captions.shape[1]) condition won't be satisfied otherwise.
        # AssertionError: The shape of the decoder output is incorrect.
        # captions.shape: torch.Size([10, 16])
        #captions = captions[:, :-1] 
        captions_length = captions.shape[1]
        captions_length_list = [captions_length for i in range(captions.shape[0])]
        caption_lengths, sort_ind = torch.FloatTensor(captions_length_list).sort(dim=0, descending=True)
        caption_lengths = (caption_lengths).tolist()
        #print('caption_lengths: ',caption_lengths)
        
        # Pass image captions through the word_embeddings layer.
        # output shape : (batch_size, caption length , embed_size)
        embeddings = self.embedding_layer(captions) # (batch_size, max_caption_length, embed_dim)
        #print ('embeddings -> (batch_size, max_caption_length, embed_dim): ', embeddings.shape)

        # Initialize LSTM state
        #h, c = self.init_decoder_state(encoder_out)  # (batch_size, decoder_dim)
        h, c = self.init_decoder_state(encoder_out)  # (batch_size, decoder_dim)
        #print('h.shape -> (batch_size, decoder_dim): ',h.shape)
        #print('c.shape -> (batch_size, decoder_dim): ',c.shape)

        # Create tensors to hold word predicion scores and alphas
        predictions = torch.zeros(batch_size, int(max(caption_lengths)), vocab_size).to(device)
        alphas = torch.zeros(batch_size, int(max(caption_lengths))).to(device)
        #print('predictions.shape: ', predictions.shape)
        #print('alphas.shape: ', alphas.shape)

        # At each time-step, decode by
        # attention-weighing the encoder's output based on the decoder's previous hidden state output
        # then generate a new word in the decoder with the previous word and the attention weighted encoding
        for t in range(int(max(caption_lengths))):
            batch_size_t = sum([l > t for l in caption_lengths])
            #print('Ciclo For nro: ', t)
            #print('batch_size_t: ', batch_size_t)
            attention_weighted_encoding, alpha = self.attention_layer(encoder_out, h)

            #gate = self.sigmoid(self.f_beta(h[:batch_size_t]))  # gating scalar, (batch_size_t, encoder_dim)
            #attention_weighted_encoding = gate * attention_weighted_encoding

            #print('attention_weighted_encoding.shape: ', attention_weighted_encoding.shape)
            #print('alpha.shape: ', alpha.shape)
            #print('embeddings[:, t, :].shape: ', embeddings[:, t, :].shape)
            #print('embeddings.shape: ', embeddings.shape)
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            #print('embeddings[:, t, :] + attention_weighted_encoding: ', torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).shape)
            #print('embeddings + attention_weighted_encoding: ', torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).shape)
            
            gate = self.sigmoid(self.f_beta(h))  # gating scalar, (batch_size_t, encoder_dim)
            attention_weighted_encoding = gate * attention_weighted_encoding
            
            # TODO: problema con c, viene creato come torch.Size([10, 512]) e poi diventa tupla dopo questa chiamata 
            #x, (h, c) = self.lstm_layer(torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1).unsqueeze(1), (h.unsqueeze(0), c.unsqueeze(0)))  # (batch_size_t, decoder_dim)
            h, c = self.lstm_layer(torch.cat([embeddings[:, t, :], attention_weighted_encoding], dim=1), (h, c))  # (batch_size_t, decoder_dim)
            #print('x.shape: ', x.shape)
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            h = h.squeeze()
            c = c.squeeze()
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            #print('c: ', c)

            preds = self.linear_fc_layer(h)  # (batch_size_t, vocab_size)
            #print('preds.shape: ', preds.shape)
            #print('alpha.shape: ', alpha.shape)
            #preds = self.fc(self.dropout(h))  # (batch_size_t, vocab_size)
            #predictions[:batch_size_t, t, :] = preds
            #alphas[:batch_size_t, t, :] = alpha
            predictions[:, t, :] = preds
            alphas[:, t] = alpha.squeeze()
            #print('predictions.shape: ', predictions.shape)
            #print('alphas.shape: ', alphas.shape)
        
        return predictions #, captions, captions_length_list, alphas

    def sample(self, inputs, states=None, max_len=20):
        " accepts pre-processed image tensor (inputs) and returns predicted sentence (list of tensor ids of length max_len) "
        pass


    def beam_search_predictions(self, encoder_out, decoder, device, vocab, beam_index = 3, max_len=20):
        word_start = vocab.start_word
        word_end = vocab.end_word
        start = vocab(word_start)
        end = vocab(word_end)
        #print('start: ', start)
        #print('end: ', end)
        vocab_size = len(vocab)

        # We'll treat the problem as having a batch size of beam_index
        #print('encoder_out.shape: ', encoder_out.shape)
        enc_image_size = encoder_out.size(1)
        #print('enc_image_size: ', enc_image_size)
        encoder_dim = encoder_out.size(2)
        #print('encoder_dim: ', encoder_dim)
        encoder_out_new = encoder_out.expand(beam_index, 1, encoder_dim)  # (beam_index, encoder_dim)
        print('encoder_out.shape: ', encoder_out.shape)

        # Tensor to store top beam_index sequences' scores; now they're just 0
        top_beam_index_scores = torch.zeros(beam_index, 1).to(device)  # (beam_index, 1)

        # Tensor to store top beam_index previous words at each step; now they're just <start>
        beam_index_prev_words = torch.LongTensor([[start]] * beam_index).to(device)  # (beam_index, 1)
        print('beam_index_prev_words.shape: ', beam_index_prev_words.shape)
        
        # Tensor to store top k sequences; now they're just <start>
        seqs = beam_index_prev_words  # (beam_index, 1)
        # Tensor to store top k sequences' alphas; now they're just 1s
        #seqs_alpha = torch.ones(beam_index, 1, enc_image_size, enc_image_size).to(device)  # (beam_index, 1, enc_image_size, enc_image_size)

        # Lists to store completed sequences, their alphas and scores
        complete_seqs = list()
        #complete_seqs_alpha = list()
        complete_seqs_scores = list()
    
        # Start decoding
        step = 1
        ## Inizialize the Hodden and Cell State
        h, c = decoder.init_decoder_state(encoder_out_new)
        h = h.squeeze()
        c = c.squeeze()

        while True:
            print('step: ', step)
            print('encoder_out_new.shape: ', encoder_out_new.shape)
            print('h.shape: ', h.shape)
            print('encoder_out_new min: ', encoder_out_new.min())
            print('encoder_out_new max: ', encoder_out_new.max())
            print('h min: ', h.min())
            print('h max: ', h.max())
            embeddings = decoder.embedding_layer(beam_index_prev_words).squeeze(1) # (beam_index, embed_dim)
            attention_weighted_encoding, alpha = decoder.attention_layer(encoder_out_new, h) # (beam_index, encoder_dim)
            alpha = alpha.view(-1, enc_image_size, enc_image_size)  # (beam_index, enc_image_size, enc_image_size)
            
            gate = decoder.sigmoid(decoder.f_beta(h))  # gating scalar, (s, encoder_dim)
            #print('gate.shape: ', gate.shape)
            #print('attention_weighted_encoding.shape: ', attention_weighted_encoding.shape)
            attention_weighted_encoding = gate * attention_weighted_encoding.squeeze(1)
            
            #print('embeddings.shape: ', embeddings.shape)
            #print('attention_weighted_encoding.shape: ', attention_weighted_encoding.shape)
            #print('embeddings + attention_weighted_encoding: ', torch.cat([embeddings, attention_weighted_encoding], dim=1).shape)
            #print('h.shape: ', h.shape)
            #print('c.shape: ', c.shape)
            h, c = decoder.lstm_layer(torch.cat([embeddings, attention_weighted_encoding.squeeze(1)], dim=1), (h, c))  # (beam_index, decoder_dim)
            preds = decoder.linear_fc_layer(h)
            
            preds = F.log_softmax(preds, dim=1)

            scores = top_beam_index_scores.expand_as(preds) + preds  # (beam_index, vocab_size)

            # For the first step, all k points will have the same scores (since same k previous words, h, c)
            if step == 1:
                top_beam_index_scores, top_beam_index_words = torch.topk(scores[0], beam_index, 0, True, True)  # (beam_index)
                prev_word_inds = top_beam_index_words / vocab_size  # (beam_index)
                next_word_inds = top_beam_index_words % vocab_size  # (beam_index)
            else:
                # Unroll and find top scores, and their unrolled indices
                top_beam_index_scores, top_beam_index_words = torch.topk(scores.view(-1), beam_index, 0, True, True)  # (beam_index)
                prev_word_inds = next_word_inds #top_beam_index_words / vocab_size  # (beam_index)
                next_word_inds = top_beam_index_words % vocab_size  # (beam_index)

            # Convert unrolled indices to actual indices of scores
            #prev_word_inds = next_word_inds #top_beam_index_words / vocab_size  # (beam_index)
            #next_word_inds = top_beam_index_words % vocab_size  # (beam_index)
            #print('top_beam_index_words: ', top_beam_index_words)
            #print('prev_word_inds: ', prev_word_inds)
            #print('next_word_inds: ', next_word_inds)
            
            # Which sequences are incomplete (didn't reach <end>)?
            '''print('end: ', end)
            gen = (count for count,ele in enumerate(next_word_inds) if ele != end)
            for count in gen:
                print('count: ', count)
            gen = (ele for count,ele in enumerate(next_word_inds) if ele != end)
            for ele in gen:
                print('ele: ', ele)'''
            #print('enumerate(next_word_inds): ', list(enumerate(next_word_inds)))
            incomplete_inds = [ind for ind, next_word in enumerate(next_word_inds) if next_word != end]
            complete_inds = list(set(range(len(next_word_inds))) - set(incomplete_inds))
            #print('incomplete_inds: ', incomplete_inds)
            #print('complete_inds: ', complete_inds)

            # Add new words to sequences, alphas
            #print('prev_word_inds: ', prev_word_inds)
            #print('next_word_inds: ', next_word_inds)
            #print('seqs.shape: ', seqs.shape)
            #print('seqs: ', seqs)
            #print('seqs[prev_word_inds].shape: ', seqs[prev_word_inds].shape)
            #print('next_word_inds.unsqueeze(1).shape: ', next_word_inds.unsqueeze(1).shape)
            #seqs = torch.cat([seqs[prev_word_inds], next_word_inds.unsqueeze(1)], dim=1)  # (beam_index, step+1)
            seqs = torch.cat([seqs, next_word_inds.unsqueeze(1)], dim=1)  # (beam_index, step+1)
            print('seqs.shape: ', seqs.shape)
            print('seqs: ', seqs)
            #print('seqs_alpha.shape: ', seqs_alpha.shape)
            #print('alpha[prev_word_inds].unsqueeze(1).shape: ', alpha[prev_word_inds].unsqueeze(1).shape)
            #seqs_alpha = torch.cat([seqs_alpha[prev_word_inds], alpha[prev_word_inds].unsqueeze(1)], dim=1)  # (beam_index, step+1, enc_image_size, enc_image_size)
            
            #print('seqs_alpha.shape: ', seqs_alpha.shape)
            #print('prev_word_inds: ', prev_word_inds)
            #print('alpha.unsqueeze(1).shape: ', alpha.unsqueeze(1).shape)
            #print('torch.cat([seqs_alpha, alpha.unsqueeze(1)], dim=1).shape: ', torch.cat([seqs_alpha, alpha.unsqueeze(1)], dim=1).shape)
            #seqs_alpha = torch.cat([seqs_alpha, alpha[prev_word_inds].unsqueeze(1)], dim=1)  # (beam_index, step+1, enc_image_size, enc_image_size)

            #print('seqs: ', seqs)
            #seqs = seqs[incomplete_inds]
            #seqs_alpha = seqs_alpha[incomplete_inds]
            #h = h[prev_word_inds[incomplete_inds]]
            #c = c[prev_word_inds[incomplete_inds]]
            #encoder_out = encoder_out[prev_word_inds[incomplete_inds]]
            top_beam_index_scores = top_beam_index_scores.unsqueeze(1)
            beam_index_prev_words = next_word_inds.unsqueeze(1)
            
            # Set aside complete sequences
            if len(complete_inds) > 0:
                complete_seqs.extend(seqs[complete_inds].tolist())
                #complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
                complete_seqs_scores.extend(top_beam_index_scores[complete_inds])
            beam_index -= len(complete_inds)  # reduce beam length accordingly
            
            # Proceed with incomplete sequences
            if beam_index == 0:
                break

            # Break if things have been going on too long
            if step > 50:
                complete_seqs.extend(seqs.tolist())
                #complete_seqs_alpha.extend(seqs_alpha[complete_inds].tolist())
                complete_seqs_scores.extend(top_beam_index_scores)
                break
            step += 1

        print('complete_seqs_scores: ', complete_seqs_scores)
        i = complete_seqs_scores.index(max(complete_seqs_scores))
        seq = complete_seqs[i]
        #alphas = complete_seqs_alpha[i]

        return seq #, alphas

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
#from model import EncoderCNN, DecoderRNN
#from modelAttention import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 2             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 1500          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

annotations_file:  data/annotations/captions_train2014.json
vocab_from_file:  True
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.46s)
creating index...


  0%|▏                                                                          | 709/414113 [00:00<01:05, 6270.39it/s]

index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [00:47<00:00, 8720.07it/s]


In [4]:
# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=1)
optimizer = torch.optim.Adam(params, lr=.0004)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

cuda


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [5]:
# LAOD NETWORK PREVIOUSLY TRAINED
'''
import os
import torch
#from modelAttention import EncoderCNN, DecoderRNN

# TODO #2: Specify the saved models to load.
encoder_file = 'models/encoder-1_20200925.pkl'
decoder_file = 'models/decoder-1_20200925.pkl'

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
encoder.eval()
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)
decoder.eval()

# Load the trained weights.
encoder.load_state_dict(torch.load(encoder_file, map_location=device))
decoder.load_state_dict(torch.load(decoder_file, map_location=device))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)
'''

"\nimport os\nimport torch\n#from modelAttention import EncoderCNN, DecoderRNN\n\n# TODO #2: Specify the saved models to load.\nencoder_file = 'models/encoder-1_20200925.pkl'\ndecoder_file = 'models/decoder-1_20200925.pkl'\n\n# Initialize the encoder and decoder, and set each to inference mode.\nencoder = EncoderCNN(embed_size)\nencoder.eval()\ndecoder = DecoderRNN(embed_size, hidden_size, vocab_size)\ndecoder.eval()\n\n# Load the trained weights.\nencoder.load_state_dict(torch.load(encoder_file, map_location=device))\ndecoder.load_state_dict(torch.load(decoder_file, map_location=device))\n\n# Move models to GPU if CUDA is available.\nencoder.to(device)\ndecoder.to(device)\n"

In [6]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

print(time.ctime())

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
#response = requests.request("GET", "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", headers={"Metadata-Flavor":"Google"})

encoder.train() # Turn on the train mode
decoder.train() # Turn on the train mode

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            #requests.request("POST", "https://nebula.udacity.com/api/v1/remote/keep-alive", headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # we use nn.utils.clip_grad_norm_ function to scale all the gradient together to prevent exploding
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), 0.5)
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        #print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d-%d.pkl' % (epoch, i_step)))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d-%d.pkl' % (epoch, i_step)))
            print(time.ctime())
            
    # Save the weights.
    if epoch % save_every == 0:
        print('Ending Epoch')
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))
        print(time.ctime())

# Close the training log file.
f.close()

Sat Sep 26 18:06:49 2020
Epoch [1/2], Step [1500/3236], Loss: 5.5756, Perplexity: 263.9015
Sat Sep 26 21:40:19 2020
Epoch [1/2], Step [3000/3236], Loss: 3.9838, Perplexity: 53.7219
Sun Sep 27 01:01:06 2020
Ending Epoch
Sun Sep 27 01:31:44 2020
Epoch [2/2], Step [1500/3236], Loss: 3.8528, Perplexity: 47.1244
Sun Sep 27 04:46:55 2020
Epoch [2/2], Step [3000/3236], Loss: 2.4361, Perplexity: 11.4283
Sun Sep 27 07:59:28 2020
Ending Epoch
Sun Sep 27 08:29:52 2020


In [ ]:
'''
Fri Sep 25 19:22:18 2020
Epoch [1/1], Step [100/3236], Loss: 4.9206, Perplexity: 137.0891
Fri Sep 25 19:42:52 2020
Epoch [1/1], Step [200/3236], Loss: 4.7592, Perplexity: 116.6542
Fri Sep 25 20:00:53 2020
Epoch [1/1], Step [300/3236], Loss: 4.7779, Perplexity: 118.8600
Fri Sep 25 20:15:53 2020
Epoch [1/1], Step [400/3236], Loss: 4.6758, Perplexity: 107.3144
Fri Sep 25 20:30:07 2020
Epoch [1/1], Step [500/3236], Loss: 4.5615, Perplexity: 95.7258
Fri Sep 25 20:44:02 2020
Epoch [1/1], Step [600/3236], Loss: 4.6132, Perplexity: 100.8030
Fri Sep 25 20:58:31 2020
Epoch [1/1], Step [700/3236], Loss: 4.5694, Perplexity: 96.4828
Fri Sep 25 21:14:43 2020
Epoch [1/1], Step [800/3236], Loss: 4.4790, Perplexity: 88.1437
Fri Sep 25 21:31:15 2020
Epoch [1/1], Step [900/3236], Loss: 4.5192, Perplexity: 91.7634
Fri Sep 25 21:48:29 2020
Epoch [1/1], Step [1000/3236], Loss: 4.4405, Perplexity: 84.8178
Fri Sep 25 22:05:14 2020
Epoch [1/1], Step [1100/3236], Loss: 4.5158, Perplexity: 91.4500
Fri Sep 25 22:22:07 2020
Epoch [1/1], Step [1200/3236], Loss: 4.4850, Perplexity: 88.6754
Fri Sep 25 22:36:57 2020
Epoch [1/1], Step [1300/3236], Loss: 4.3665, Perplexity: 78.7691
Fri Sep 25 22:51:43 2020
Epoch [1/1], Step [1400/3236], Loss: 4.4671, Perplexity: 87.1053
Fri Sep 25 23:05:51 2020
Epoch [1/1], Step [1500/3236], Loss: 4.2293, Perplexity: 68.6689
Fri Sep 25 23:19:50 2020
Epoch [1/1], Step [1600/3236], Loss: 4.3653, Perplexity: 78.6728
Fri Sep 25 23:33:05 2020
Epoch [1/1], Step [1700/3236], Loss: 4.1140, Perplexity: 61.1905
Fri Sep 25 23:46:15 2020
Epoch [1/1], Step [1800/3236], Loss: 4.4670, Perplexity: 87.0928
Fri Sep 25 23:59:18 2020
Epoch [1/1], Step [1900/3236], Loss: 4.6002, Perplexity: 99.5082
Sat Sep 26 00:12:22 2020
Epoch [1/1], Step [2000/3236], Loss: 4.4533, Perplexity: 85.9111
Sat Sep 26 00:25:22 2020
Epoch [1/1], Step [2100/3236], Loss: 4.4441, Perplexity: 85.1226
Sat Sep 26 00:38:21 2020
Epoch [1/1], Step [2200/3236], Loss: 4.4543, Perplexity: 85.9945
Sat Sep 26 00:51:26 2020
Epoch [1/1], Step [2300/3236], Loss: 4.2585, Perplexity: 70.7060
Sat Sep 26 01:04:26 2020
Epoch [1/1], Step [2400/3236], Loss: 4.4038, Perplexity: 81.7587
Sat Sep 26 01:18:34 2020
Epoch [1/1], Step [2500/3236], Loss: 4.2635, Perplexity: 71.0605
Sat Sep 26 01:31:39 2020
Epoch [1/1], Step [2600/3236], Loss: 4.8209, Perplexity: 124.0712
Sat Sep 26 01:45:46 2020
Epoch [1/1], Step [2700/3236], Loss: 4.6322, Perplexity: 102.7426
Sat Sep 26 01:58:50 2020
Epoch [1/1], Step [2800/3236], Loss: 4.7834, Perplexity: 119.5124
Sat Sep 26 02:11:57 2020
Epoch [1/1], Step [2900/3236], Loss: 4.7017, Perplexity: 110.1307
Sat Sep 26 02:24:54 2020
Epoch [1/1], Step [3000/3236], Loss: 4.6135, Perplexity: 100.8355
Sat Sep 26 02:37:49 2020
Epoch [1/1], Step [3100/3236], Loss: 4.5976, Perplexity: 99.2477
Sat Sep 26 02:50:52 2020
Epoch [1/1], Step [3200/3236], Loss: 4.6212, Perplexity: 101.6168
Sat Sep 26 03:03:55 2020
Ending Epoch
Sat Sep 26 03:08:40 2020
'''

In [ ]:
        #print('Inizio controlli')
        #print('outputs.shape: ', outputs.shape)
        #print('caption.shape: ', captions.shape)
        #print('outputs: ', outputs)
        
        
        #torch.set_printoptions(profile="full")
        #print('caption: ', captions)
        #print('torch.argmax(outputs, dim=2): ', torch.argmax(outputs, dim=2))
        #torch.set_printoptions(profile="default")
        
        
        #print('torch.argmax(outputs, dim=2).shape: ', torch.argmax(outputs, dim=2).shape)
        #print('torch.argmax(outputs, dim=2).view(-1).shape: ', torch.argmax(outputs, dim=2).view(-1).shape)
        #outputs = torch.argmax(outputs, dim=2)
        #captions = captions.view(-1)
        
        #print('outputs.shape: ', outputs.shape)
        #print('caption.shape: ', captions.shape)
        #torch.set_printoptions(profile="full")
        #print('outputs: ', outputs)
        #print('caption: ', captions)
        #torch.set_printoptions(profile="default")
        
        
        #loss = criterion(outputs.view(-1, vocab_size), captions)
        #loss = criterion(outputs, captions)

In [ ]:
'''
Keep up the Training
'''

In [3]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
#from model import EncoderCNN, DecoderRNN
#from modelAttention import EncoderCNN, DecoderRNN
import math


## TODO #1: Select appropriate values for the Python variables below.
batch_size = 128          # batch size
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 1             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 1500          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

annotations_file:  data/annotations/captions_train2014.json
vocab_from_file:  True
Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...
Done (t=1.46s)
creating index...


  0%|▎                                                                         | 1407/414113 [00:00<01:06, 6247.26it/s]

index created!
Obtaining caption lengths...


100%|████████████████████████████████████████████████████████████████████████| 414113/414113 [00:49<00:00, 8337.78it/s]


In [4]:
# Watch for any changes in model.py, and re-load it automatically.
#% load_ext autoreload
#% autoreload 2

import os
import torch
#from modelAttention import EncoderCNN, DecoderRNN

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("cuda" if torch.cuda.is_available() else "cpu")

# TODO #2: Specify the saved models to load.
encoder_file = 'models/encoder-2.pkl'
decoder_file = 'models/decoder-2.pkl'

# TODO #3: Select appropriate values for the Python variables below.
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder, and set each to inference mode.
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size)

# Load the trained weights.
encoder.load_state_dict(torch.load(encoder_file, map_location=device))
decoder.load_state_dict(torch.load(decoder_file, map_location=device))

# Move models to GPU if CUDA is available.
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=1)
optimizer = torch.optim.Adam(params, lr=.0001)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

cuda


In [5]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

print(time.ctime())

# Open the training log file.
f = open(log_file, 'w')

old_time = time.time()
#response = requests.request("GET", "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", headers={"Metadata-Flavor":"Google"})

encoder.train() # Turn on the train mode
decoder.train() # Turn on the train mode

for epoch in range(1, num_epochs+1):
    
    for i_step in range(1, total_step+1):
        
        if time.time() - old_time > 60:
            old_time = time.time()
            #requests.request("POST", "https://nebula.udacity.com/api/v1/remote/keep-alive", headers={'Authorization': "STAR " + response.text})
        
        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler
        
        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)
        
        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()
        
        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)
        
        # Calculate the batch loss.
        loss = criterion(outputs.view(-1, vocab_size), captions.view(-1))
        
        # Backward pass.
        loss.backward()
        
        # we use nn.utils.clip_grad_norm_ function to scale all the gradient together to prevent exploding
        torch.nn.utils.clip_grad_norm_(decoder.parameters(), 0.5)
        
        # Update the parameters in the optimizer.
        optimizer.step()
            
        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))
        
        # Print training statistics (on same line).
        #print('\r' + stats, end="")
        sys.stdout.flush()
        
        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()
        
        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)
            torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d-%d-trn-20200928.pkl' % (epoch, i_step)))
            torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d-%d-trn-20200928.pkl' % (epoch, i_step)))
            print(time.ctime())
            
    # Save the weights.
    if epoch % save_every == 0:
        print('Ending Epoch')
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d-trn-20200928.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d-trn-20200928.pkl' % epoch))
        print(time.ctime())

# Close the training log file.
f.close()

Mon Sep 28 22:27:20 2020


RuntimeError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 1.60 GiB already allocated; 311.30 MiB free; 437.96 MiB cached)

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.